# Monte Carlo Walk-Forward Validation - Statistical Arbitrage Strategy

**Strategy**: Hypothesis 5 - Statistical Arbitrage Pairs Trading  
**Project ID**: 26140717  
**Optimized Sharpe**: 1.829  
**Baseline Sharpe**: 0.127  

## Optimized Parameters to Validate:
- z_entry_threshold: 1.5
- z_exit_threshold: 1.0
- lookback_period: 30
- position_size_per_pair: 0.40
- max_holding_days: 30
- stop_loss_z: 4.0

## Approach:
Uses QuantBook to:
1. Access historical data for pairs (PNC/KBE, ARCC/AMLP, RBA/SMFG, ENB/WEC)
2. Run Monte Carlo splits (random train/test periods)
3. Execute strategy logic locally in Python
4. Calculate Sharpe ratio for each period
5. Analyze degradation (train vs test)

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
from collections import Counter, deque
import json

# QuantConnect Research
from QuantConnect import *
from QuantConnect.Research import QuantBook

# Initialize QuantBook
qb = QuantBook()

print("✓ QuantConnect Research environment initialized")

✓ QuantConnect Research environment initialized


In [2]:
# ==================== CONFIGURATION ====================

config = {
    'project_id': 26140717,
    
    # Pairs to trade
    'pairs': [
        {'long': 'PNC', 'short': 'KBE', 'name': 'PNC_KBE'},
        {'long': 'ARCC', 'short': 'AMLP', 'name': 'ARCC_AMLP'},
        {'long': 'RBA', 'short': 'SMFG', 'name': 'RBA_SMFG'},
        {'long': 'ENB', 'short': 'WEC', 'name': 'ENB_WEC'}
    ],
    
    # Total period for analysis
    'total_period': {
        'start': datetime(2022, 1, 1),
        'end': datetime(2024, 12, 31)  # Use only historical data (no future dates)
    },
    
    # Monte Carlo configuration
    'train_test_split': 0.70,
    'monte_carlo_runs': 100,  # Gradual scaling: 20 → 50 → 100 → ... → 1000+
    'random_seed': 42,
    
    # Optimized parameters to test
    'parameters': {
        'z_entry_threshold': 1.5,
        'z_exit_threshold': 1.0,
        'lookback_period': 30,
        'position_size_per_pair': 0.40,
        'max_holding_days': 30,
        'stop_loss_z': 4.0
    },
    
    'baseline_sharpe': 1.829,
    'initial_capital': 100000
}

# Set random seed
if config['random_seed']:
    random.seed(config['random_seed'])
    np.random.seed(config['random_seed'])

print("Configuration:")
print(f"  Pairs: {len(config['pairs'])}")
print(f"  Period: {config['total_period']['start'].date()} to {config['total_period']['end'].date()}")
print(f"  Train/Test: {config['train_test_split']*100:.0f}%/{(1-config['train_test_split'])*100:.0f}%")
print(f"  Monte Carlo runs: {config['monte_carlo_runs']} (testing gradually toward 1000+)")
print(f"  Parameters: {config['parameters']}")
print(f"  Baseline Sharpe: {config['baseline_sharpe']:.3f}")

Configuration:
  Pairs: 4
  Period: 2022-01-01 to 2024-12-31
  Train/Test: 70%/30%
  Monte Carlo runs: 20 (testing gradually toward 1000+)
  Parameters: {'z_entry_threshold': 1.5, 'z_exit_threshold': 1.0, 'lookback_period': 30, 'position_size_per_pair': 0.4, 'max_holding_days': 30, 'stop_loss_z': 4.0}
  Baseline Sharpe: 1.829


In [3]:
# ==================== SUBSCRIBE TO SECURITIES ====================

print("Subscribing to securities...")

symbols = {}
for pair in config['pairs']:
    long_sym = qb.AddEquity(pair['long'], Resolution.Daily).Symbol
    short_sym = qb.AddEquity(pair['short'], Resolution.Daily).Symbol
    symbols[pair['name']] = {'long': long_sym, 'short': short_sym}
    print(f"  ✓ {pair['name']}: {pair['long']}/{pair['short']}")

print(f"\n✓ Subscribed to {len(symbols)} pairs")

Subscribing to securities...
  ✓ PNC_KBE: PNC/KBE
  ✓ ARCC_AMLP: ARCC/AMLP
  ✓ RBA_SMFG: RBA/SMFG
  ✓ ENB_WEC: ENB/WEC

✓ Subscribed to 4 pairs


In [ ]:
# ==================== HELPER FUNCTIONS ====================

def generate_random_split(start_date, end_date, train_pct, seed=None):
    """Generate random train/test split for Monte Carlo - GUARANTEES test_end <= end_date

    Strategy: Work backwards from end_date to ensure test period never exceeds boundary.
    We randomly position a combined train+test window within the available range,
    but ALWAYS anchor the test period to end no later than end_date.
    """
    # Seed parameter kept for API compatibility but not used
    # MC runs should use sequential random state, not reset per call

    total_days = (end_date - start_date).days
    train_days = int(total_days * train_pct)
    test_days = total_days - train_days

    # Ensure minimum test period (60 calendar days ≈ 40-45 trading days)
    min_test_days = 60
    if test_days < min_test_days:
        test_days = min_test_days
        train_days = total_days - test_days

    # NEW APPROACH: Position the test period randomly, but ALWAYS end at or before end_date
    # Test period can start anywhere from (start_date + train_days + 1) to (end_date - test_days)
    earliest_test_start = start_date + timedelta(days=train_days + 1)
    latest_test_start = end_date - timedelta(days=test_days)

    if latest_test_start < earliest_test_start:
        # Not enough room - use sequential split
        train_start = start_date
        train_end = start_date + timedelta(days=train_days)
        test_start = train_end + timedelta(days=1)
        test_end = end_date  # GUARANTEED to be end_date
    else:
        # Random positioning of test start (but test always ends at or before end_date)
        days_range = (latest_test_start - earliest_test_start).days
        random_offset = random.randint(0, max(0, days_range))

        test_start = earliest_test_start + timedelta(days=random_offset)
        test_end = test_start + timedelta(days=test_days)

        # CRITICAL: Ensure test_end does not exceed end_date
        if test_end > end_date:
            test_end = end_date
            test_start = test_end - timedelta(days=test_days)

        # Calculate training period (ends day before test starts)
        train_end = test_start - timedelta(days=1)
        train_start = train_end - timedelta(days=train_days)

        # Ensure train_start doesn't go before start_date
        if train_start < start_date:
            train_start = start_date
            train_end = test_start - timedelta(days=1)

    # Final validation
    assert test_end <= end_date, f"BUG: test_end {test_end} exceeds end_date {end_date}"
    assert train_start >= start_date, f"BUG: train_start {train_start} before start_date {start_date}"

    return train_start, train_end, test_start, test_end


def calculate_spread(long_prices, short_prices):
    """Calculate spread between two price series"""
    return np.log(long_prices) - np.log(short_prices)


def calculate_zscore(spread, lookback):
    """Calculate z-score using rolling window"""
    if len(spread) < lookback:
        return pd.Series([np.nan] * len(spread), index=spread.index)
    
    rolling_mean = spread.rolling(window=lookback).mean()
    rolling_std = spread.rolling(window=lookback).std(ddof=1)
    
    zscore = (spread - rolling_mean) / rolling_std
    return zscore


def simulate_strategy(data, params):
    """
    Simulate statistical arbitrage strategy on historical data
    
    Args:
        data: Dict of DataFrames with price data for each pair
        params: Strategy parameters
    
    Returns:
        equity_curve: Daily portfolio values
        trades: List of trade records
    """
    capital = config['initial_capital']
    equity_curve = []
    trades = []
    
    # Get all dates (union of all pair dates)
    all_dates = sorted(set().union(*[set(df.index) for df in data.values()]))
    
    # Track positions for each pair
    positions = {pair['name']: None for pair in config['pairs']}
    
    for date in all_dates:
        daily_pnl = 0
        
        # Process each pair
        for pair in config['pairs']:
            pair_name = pair['name']
            if pair_name not in data:
                continue
                
            df = data[pair_name]
            
            if date not in df.index:
                continue
            
            # Get current prices and z-score
            current_data = df.loc[:date]
            if len(current_data) < params['lookback_period']:
                continue
            
            long_price = df.loc[date, 'long_price']
            short_price = df.loc[date, 'short_price']
            z_score = df.loc[date, 'zscore']
            
            if np.isnan(z_score):
                continue
            
            pos = positions[pair_name]
            
            # Check exit conditions
            if pos is not None:
                days_held = (date - pos['entry_date']).days
                
                # Calculate current P&L
                if pos['direction'] == 'long_spread':
                    pnl = (long_price / pos['entry_long'] - 1) * pos['long_shares'] * pos['entry_long']
                    pnl -= (short_price / pos['entry_short'] - 1) * pos['short_shares'] * pos['entry_short']
                else:
                    pnl = (short_price / pos['entry_short'] - 1) * pos['short_shares'] * pos['entry_short']
                    pnl -= (long_price / pos['entry_long'] - 1) * pos['long_shares'] * pos['entry_long']
                
                daily_pnl += pnl - pos['last_pnl']
                pos['last_pnl'] = pnl
                
                # Exit conditions
                exit_signal = False
                exit_reason = None
                
                if abs(z_score) < params['z_exit_threshold']:
                    exit_signal = True
                    exit_reason = 'mean_reversion'
                elif days_held >= params['max_holding_days']:
                    exit_signal = True
                    exit_reason = 'timeout'
                elif abs(z_score) > params['stop_loss_z']:
                    exit_signal = True
                    exit_reason = 'stop_loss'
                
                if exit_signal:
                    capital += pnl
                    trades.append({
                        'pair': pair_name,
                        'entry_date': pos['entry_date'],
                        'exit_date': date,
                        'entry_z': pos['entry_z'],
                        'exit_z': z_score,
                        'pnl': pnl,
                        'exit_reason': exit_reason,
                        'days_held': days_held
                    })
                    positions[pair_name] = None
            
            # Check entry conditions (if no position)
            if positions[pair_name] is None:
                if abs(z_score) > params['z_entry_threshold']:
                    # Calculate position sizes (dollar-neutral)
                    pair_capital = capital * params['position_size_per_pair']
                    
                    if z_score > 0:  # Short spread (long short, short long)
                        direction = 'short_spread'
                        long_shares = pair_capital / (2 * long_price)
                        short_shares = pair_capital / (2 * short_price)
                    else:  # Long spread (long long, short short)
                        direction = 'long_spread'
                        long_shares = pair_capital / (2 * long_price)
                        short_shares = pair_capital / (2 * short_price)
                    
                    positions[pair_name] = {
                        'entry_date': date,
                        'entry_z': z_score,
                        'entry_long': long_price,
                        'entry_short': short_price,
                        'long_shares': long_shares,
                        'short_shares': short_shares,
                        'direction': direction,
                        'last_pnl': 0
                    }
        
        # Record equity
        equity_curve.append({'date': date, 'equity': capital})
    
    return pd.DataFrame(equity_curve).set_index('date'), trades


def calculate_sharpe(equity_curve):
    """Calculate annualized Sharpe ratio"""
    returns = equity_curve['equity'].pct_change().dropna()
    if len(returns) == 0 or returns.std() == 0:
        return 0.0
    
    sharpe = returns.mean() / returns.std() * np.sqrt(252)  # Annualized
    return sharpe


print("✓ Helper functions loaded")

In [ ]:
# ==================== ADVANCED MONTE CARLO METRICS ====================

from scipy import stats
from scipy.special import ndtr  # Standard normal CDF

def calculate_return_statistics(equity_curve):
    """Calculate comprehensive return statistics including higher moments"""
    returns = equity_curve['equity'].pct_change().dropna()
    
    if len(returns) == 0:
        return {
            'mean': 0, 'std': 0, 'skewness': 0, 'kurtosis': 3,
            'sharpe': 0, 'observations': 0
        }
    
    return {
        'mean': float(returns.mean()),
        'std': float(returns.std()),
        'skewness': float(stats.skew(returns)),
        'kurtosis': float(stats.kurtosis(returns, fisher=False)),  # Pearson kurtosis (normal = 3)
        'sharpe': float(returns.mean() / returns.std() * np.sqrt(252)) if returns.std() > 0 else 0,
        'observations': len(returns)
    }


def calculate_probabilistic_sharpe_ratio(sharpe, returns_stats, benchmark_sr=0.0):
    """
    Calculate Probabilistic Sharpe Ratio (PSR)
    
    PSR represents the probability that the true Sharpe ratio exceeds a benchmark.
    Industry threshold: PSR ≥ 0.95 for statistical significance at 95% confidence.
    
    Args:
        sharpe: Estimated Sharpe ratio
        returns_stats: Dict with 'observations', 'skewness', 'kurtosis'
        benchmark_sr: Benchmark Sharpe ratio to test against (default 0.0)
    
    Returns:
        PSR value between 0 and 1
    
    Reference: Bailey & López de Prado (2012) "The Sharpe Ratio Efficient Frontier"
    """
    T = returns_stats['observations']
    skew = returns_stats['skewness']
    kurt = returns_stats['kurtosis']
    
    if T < 2:
        return 0.0
    
    # Standard error of Sharpe ratio (accounting for non-normality)
    # σ̂(ŜR) = √[(1 + (SR²/2) - γ₃·SR + ((γ₄-3)/4)·SR²) / (T-1)]
    variance_term = 1 + (sharpe**2 / 2) - skew * sharpe + ((kurt - 3) / 4) * sharpe**2
    variance_term = max(variance_term, 0.0001)  # Prevent negative variance
    
    se_sharpe = np.sqrt(variance_term / (T - 1))
    
    # PSR = Φ[(ŜR - SR*) / σ̂(ŜR)]
    # where Φ is the standard normal CDF
    if se_sharpe > 0:
        z_score = (sharpe - benchmark_sr) / se_sharpe
        psr = ndtr(z_score)  # Standard normal CDF
    else:
        psr = 1.0 if sharpe > benchmark_sr else 0.0
    
    return float(psr)


def calculate_minimum_track_record_length(sharpe, returns_stats, benchmark_sr=0.0, confidence=0.95):
    """
    Calculate Minimum Track Record Length (MinTRL)
    
    MinTRL is the time period required to reject the null hypothesis that SR ≤ SR*
    with specified confidence level.
    
    Args:
        sharpe: Estimated Sharpe ratio
        returns_stats: Dict with 'skewness', 'kurtosis'
        benchmark_sr: Benchmark Sharpe ratio
        confidence: Confidence level (default 0.95 for 95%)
    
    Returns:
        Minimum number of observations required
    
    Reference: Bailey & López de Prado (2014) "The Deflated Sharpe Ratio"
    """
    if sharpe <= benchmark_sr:
        return float('inf')  # Cannot achieve significance if SR ≤ SR*
    
    skew = returns_stats['skewness']
    kurt = returns_stats['kurtosis']
    
    # Z-score for confidence level
    z_alpha = stats.norm.ppf(confidence)
    
    # MinTRL = 1 + [1 - γ₃×SR + (γ₄-1)/4 × SR²] × (Z_α/(SR-SR*))²
    factor = 1 - skew * sharpe + ((kurt - 1) / 4) * sharpe**2
    factor = max(factor, 0.0001)
    
    min_trl = 1 + factor * (z_alpha / (sharpe - benchmark_sr))**2
    
    return int(np.ceil(min_trl))


def calculate_deflated_sharpe_ratio(sharpe_ratios, confidence=0.95):
    """
    Calculate Deflated Sharpe Ratio (DSR) to correct for multiple testing
    
    When testing N strategies, DSR adjusts for the probability of finding
    lucky results purely by chance.
    
    Args:
        sharpe_ratios: List/array of Sharpe ratios from multiple tests
        confidence: Confidence level (default 0.95)
    
    Returns:
        DSR value (probability of genuine edge after multiple testing)
    
    Reference: Bailey & López de Prado (2014)
    """
    sharpe_array = np.array(sharpe_ratios)
    N = len(sharpe_array)
    
    if N <= 1:
        return 1.0  # No multiple testing correction needed
    
    # Variance of Sharpe ratios
    var_sharpes = np.var(sharpe_array, ddof=1)
    
    if var_sharpes == 0:
        return 1.0
    
    # Euler-Mascheroni constant
    gamma_em = 0.5772156649
    
    # Expected maximum Sharpe under null hypothesis
    # SR₀* = √Var[{ŜRₙ}] × [(1-γ)·Φ⁻¹[1-1/N] + γ·Φ⁻¹[1-(1/N)·e⁻¹]]
    term1 = (1 - gamma_em) * stats.norm.ppf(1 - 1/N)
    term2 = gamma_em * stats.norm.ppf(1 - (1/N) * np.exp(-1))
    
    sr_expected_max = np.sqrt(var_sharpes) * (term1 + term2)
    
    # Best observed Sharpe
    sr_best = np.max(sharpe_array)
    
    # Calculate PSR using expected max as benchmark
    # Simplified: assuming normal distribution for DSR calculation
    if sr_best > sr_expected_max:
        z_score = (sr_best - sr_expected_max) / np.sqrt(var_sharpes / N)
        dsr = ndtr(z_score)
    else:
        dsr = 0.5  # No better than random
    
    return float(dsr)


def analyze_mc_distribution(values):
    """
    Analyze distribution of Monte Carlo results
    
    Returns comprehensive statistics including percentiles, which are
    more robust than mean for non-normal distributions.
    
    Args:
        values: Array of values from MC simulations
    
    Returns:
        Dict with distribution statistics
    """
    arr = np.array(values)
    
    if len(arr) == 0:
        return {
            'mean': 0, 'median': 0, 'std': 0,
            'p10': 0, 'p25': 0, 'p75': 0, 'p90': 0,
            'min': 0, 'max': 0, 'skewness': 0, 'kurtosis': 3
        }
    
    return {
        'mean': float(np.mean(arr)),
        'median': float(np.median(arr)),
        'std': float(np.std(arr, ddof=1)),
        'p10': float(np.percentile(arr, 10)),   # Worst case in 90% of scenarios
        'p25': float(np.percentile(arr, 25)),
        'p75': float(np.percentile(arr, 75)),
        'p90': float(np.percentile(arr, 90)),
        'min': float(np.min(arr)),
        'max': float(np.max(arr)),
        'skewness': float(stats.skew(arr)),
        'kurtosis': float(stats.kurtosis(arr, fisher=False))
    }


def calculate_confidence_interval(values, confidence=0.95):
    """Calculate confidence interval for a set of values"""
    arr = np.array(values)
    if len(arr) < 2:
        return (0, 0)
    
    mean = np.mean(arr)
    se = stats.sem(arr)  # Standard error of mean
    z_score = stats.norm.ppf((1 + confidence) / 2)
    
    ci_lower = mean - z_score * se
    ci_upper = mean + z_score * se
    
    return (float(ci_lower), float(ci_upper))


def assess_sample_size_adequacy(n_trades, autocorrelation=0.0):
    """
    Assess whether sample size is adequate for statistical validation
    
    Args:
        n_trades: Number of trades
        autocorrelation: Average return correlation (0 = independent)
    
    Returns:
        Dict with assessment
    """
    # Adjust for autocorrelation
    # n_effective = n_actual × (1 - ρ)
    n_effective = n_trades * (1 - abs(autocorrelation))
    
    # Thresholds
    if n_effective >= 1000:
        adequacy = "EXCELLENT"
        confidence = "High confidence in statistical inference"
    elif n_effective >= 100:
        adequacy = "GOOD"
        confidence = "Adequate for production systems"
    elif n_effective >= 50:
        adequacy = "MARGINAL"
        confidence = "Minimum acceptable, use with caution"
    elif n_effective >= 30:
        adequacy = "WEAK"
        confidence = "Preliminary hypothesis testing only"
    else:
        adequacy = "INSUFFICIENT"
        confidence = "Too few samples for reliable inference"
    
    return {
        'n_actual': n_trades,
        'n_effective': int(n_effective),
        'autocorrelation': autocorrelation,
        'adequacy': adequacy,
        'confidence': confidence,
        'recommended_minimum': 100
    }


print("✓ Advanced Monte Carlo metrics functions loaded")
print("  - Probabilistic Sharpe Ratio (PSR)")
print("  - Deflated Sharpe Ratio (DSR)")
print("  - Minimum Track Record Length (MinTRL)")
print("  - Distribution analysis with percentiles")
print("  - Sample size adequacy assessment")


In [5]:
# ==================== MONTE CARLO WALK-FORWARD ====================

print("="*70)
print("MONTE CARLO WALK-FORWARD ANALYSIS - STATISTICAL ARBITRAGE")
print("="*70)
print()

results = []
errors = []

for run in range(config['monte_carlo_runs']):
    print(f"\n{'='*70}")
    print(f"Monte Carlo Run {run + 1}/{config['monte_carlo_runs']}")
    print(f"{'='*70}")
    
    try:
        # 1. Generate random train/test split
        train_start, train_end, test_start, test_end = generate_random_split(
            config['total_period']['start'],
            config['total_period']['end'],
            config['train_test_split'],
            seed=(config['random_seed'] + run) if config['random_seed'] else None
        )
        
        print(f"Training:  {train_start.date()} to {train_end.date()} ({(train_end - train_start).days} days)")
        print(f"Testing:   {test_start.date()} to {test_end.date()} ({(test_end - test_start).days} days)")
        
        # 2. Fetch historical data for TRAINING period
        print(f"\nFetching training data...")
        train_data = {}
        for pair in config['pairs']:
            # Fetch history - use list with single symbol to get clean DataFrame
            long_hist = qb.History([symbols[pair['name']]['long']], train_start, train_end, Resolution.Daily)
            short_hist = qb.History([symbols[pair['name']]['short']], train_start, train_end, Resolution.Daily)
            
            if long_hist.empty or short_hist.empty:
                print(f"  ⚠ Skipping {pair['name']}: no data")
                continue
            
            # Extract close prices - handle multi-index if present
            if isinstance(long_hist.index, pd.MultiIndex):
                long_close = long_hist['close'].droplevel(0)
                short_close = short_hist['close'].droplevel(0)
            else:
                long_close = long_hist['close']
                short_close = short_hist['close']
            
            # Create aligned DataFrame
            df = pd.DataFrame({
                'long_price': long_close,
                'short_price': short_close
            }).dropna()
            
            # Only require lookback period worth of data
            if len(df) < config['parameters']['lookback_period']:
                print(f"  ⚠ Skipping {pair['name']}: insufficient data ({len(df)} rows, need {config['parameters']['lookback_period']})")
                continue
            
            # Calculate spread and z-score
            df['spread'] = np.log(df['long_price']) - np.log(df['short_price'])
            df['zscore'] = calculate_zscore(df['spread'], config['parameters']['lookback_period'])
            
            train_data[pair['name']] = df
            print(f"  ✓ {pair['name']}: {len(df)} days")
        
        if len(train_data) == 0:
            raise ValueError("No training data available for any pair")
        
        print(f"  ✓ Fetched data for {len(train_data)} pairs")
        
        # 3. Run strategy on TRAINING data
        print(f"Running strategy on training period...")
        train_equity, train_trades = simulate_strategy(train_data, config['parameters'])
        train_sharpe = calculate_sharpe(train_equity)
        print(f"  ✓ Training Sharpe: {train_sharpe:.3f} ({len(train_trades)} trades)")
        
        # 4. Fetch historical data for TESTING period
        print(f"\nFetching testing data...")
        test_data = {}
        for pair in config['pairs']:
            # Fetch history
            long_hist = qb.History([symbols[pair['name']]['long']], test_start, test_end, Resolution.Daily)
            short_hist = qb.History([symbols[pair['name']]['short']], test_start, test_end, Resolution.Daily)
            
            if long_hist.empty or short_hist.empty:
                print(f"  ⚠ Skipping {pair['name']}: no data")
                continue
            
            # Extract close prices
            if isinstance(long_hist.index, pd.MultiIndex):
                long_close = long_hist['close'].droplevel(0)
                short_close = short_hist['close'].droplevel(0)
            else:
                long_close = long_hist['close']
                short_close = short_hist['close']
            
            # Create aligned DataFrame
            df = pd.DataFrame({
                'long_price': long_close,
                'short_price': short_close
            }).dropna()
            
            # Only require lookback period worth of data
            if len(df) < config['parameters']['lookback_period']:
                print(f"  ⚠ Skipping {pair['name']}: insufficient data ({len(df)} rows, need {config['parameters']['lookback_period']})")
                continue
            
            # Calculate spread and z-score
            df['spread'] = np.log(df['long_price']) - np.log(df['short_price'])
            df['zscore'] = calculate_zscore(df['spread'], config['parameters']['lookback_period'])
            
            test_data[pair['name']] = df
            print(f"  ✓ {pair['name']}: {len(df)} days")
        
        if len(test_data) == 0:
            raise ValueError("No testing data available for any pair")
        
        print(f"  ✓ Fetched data for {len(test_data)} pairs")
        
        # 5. Run strategy on TESTING data
        print(f"Running strategy on testing period...")
        test_equity, test_trades = simulate_strategy(test_data, config['parameters'])
        test_sharpe = calculate_sharpe(test_equity)
        print(f"  ✓ Testing Sharpe: {test_sharpe:.3f} ({len(test_trades)} trades)")

        # 5a. Calculate return statistics for advanced metrics
        train_stats = calculate_return_statistics(train_equity)
        test_stats = calculate_return_statistics(test_equity)

        # Calculate PSR for test period
        test_psr = calculate_probabilistic_sharpe_ratio(
            test_sharpe,
            test_stats,
            benchmark_sr=0.0  # Test against SR > 0
        )

        # Calculate MinTRL
        min_trl = calculate_minimum_track_record_length(
            test_sharpe,
            test_stats,
            benchmark_sr=0.0
        ) if test_sharpe > 0 else float('inf')

        print(f"  PSR: {test_psr:.3f}, MinTRL: {min_trl if min_trl != float('inf') else 'N/A'}")
        
        # 6. Calculate degradation
        if train_sharpe > 0:
            degradation = (train_sharpe - test_sharpe) / train_sharpe
        else:
            degradation = 1.0
        
        print(f"  Degradation: {degradation*100:.1f}%")
        
        # Store results
        results.append({
            'run': run + 1,
            'train_start': train_start,
            'train_end': train_end,
            'test_start': test_start,
            'test_end': test_end,
            'train_sharpe': float(train_sharpe),
            'test_sharpe': float(test_sharpe),
            'degradation': float(degradation),
            'train_trades': len(train_trades),
            'test_trades': len(test_trades),
            # Advanced metrics
            'test_psr': float(test_psr),
            'test_skewness': test_stats['skewness'],
            'test_kurtosis': test_stats['kurtosis'],
            'test_observations': test_stats['observations'],
            'min_trl': min_trl if min_trl != float('inf') else None,
            'train_skewness': train_stats['skewness'],
            'train_kurtosis': train_stats['kurtosis'],
            'train_observations': train_stats['observations'],
        })
        
        print(f"  ✓ Run {run + 1} complete")
        
    except Exception as e:
        import traceback
        error_msg = str(e)
        traceback_str = traceback.format_exc()
        print(f"  ✗ Error in run {run + 1}: {error_msg}")
        print(f"  Traceback:\n{traceback_str}")
        errors.append({'run': run + 1, 'error': error_msg, 'traceback': traceback_str})
        continue

print(f"\n{'='*70}")
print(f"Monte Carlo Walk-Forward Complete")
print(f"  Successful runs: {len(results)}/{config['monte_carlo_runs']}")
print(f"  Failed runs: {len(errors)}/{config['monte_carlo_runs']}")
print(f"{'='*70}")

MONTE CARLO WALK-FORWARD ANALYSIS - STATISTICAL ARBITRAGE


Monte Carlo Run 1/20
Training:  2022-07-17 to 2024-08-21 (766 days)
Testing:   2024-08-22 to 2025-07-16 (328 days)

Fetching training data...
  ⚠ Skipping PNC_KBE: no data
  ⚠ Skipping ARCC_AMLP: no data
  ⚠ Skipping RBA_SMFG: no data
  ⚠ Skipping ENB_WEC: no data


ValueError: No training data available for any pair

In [ ]:
# ==================== ANALYSIS ====================

if len(results) == 0:
    print("✗ No successful runs to analyze")
else:
    df_results = pd.DataFrame(results)
    
    print("\n" + "="*70)
    print("AGGREGATE RESULTS")
    print("="*70)
    
    # Basic statistics
    mean_train = df_results['train_sharpe'].mean()
    std_train = df_results['train_sharpe'].std()
    mean_test = df_results['test_sharpe'].mean()
    std_test = df_results['test_sharpe'].std()
    mean_deg = df_results['degradation'].mean()
    std_deg = df_results['degradation'].std()
    
    print(f"\nPerformance Metrics:")
    print(f"  Baseline Sharpe (original):  {config['baseline_sharpe']:.3f}")
    print(f"  Mean Training Sharpe:         {mean_train:.3f} ± {std_train:.3f}")
    print(f"  Mean Testing Sharpe:          {mean_test:.3f} ± {std_test:.3f}")
    print(f"  Mean Degradation:             {mean_deg*100:.1f}% ± {std_deg*100:.1f}%")
    
    # ==================== ADVANCED MONTE CARLO METRICS ====================
    
    print(f"\n" + "="*70)
    print("ADVANCED MONTE CARLO METRICS")
    print("="*70)
    
    # PSR Analysis
    test_psrs = [r['test_psr'] for r in results]
    psr_dist = analyze_mc_distribution(test_psrs)
    
    print(f"\n📊 Probabilistic Sharpe Ratio (PSR):")
    print(f"   Median PSR: {psr_dist['median']:.3f}")
    print(f"   10th Percentile: {psr_dist['p10']:.3f} ← Use for conservative planning")
    print(f"   90th Percentile: {psr_dist['p90']:.3f}")
    print(f"   Status: {'✓ PASS' if psr_dist['p10'] >= 0.95 else '⚠ MARGINAL' if psr_dist['p10'] >= 0.90 else '✗ FAIL'}")
    print(f"   (Threshold: 10th %ile PSR ≥ 0.95 for statistical significance)")
    
    # Test Sharpe Distribution
    test_sharpe_dist = analyze_mc_distribution(df_results['test_sharpe'])
    print(f"\n📈 Test Sharpe Distribution:")
    print(f"   10th Percentile: {test_sharpe_dist['p10']:.3f}")
    print(f"   Median: {test_sharpe_dist['median']:.3f}")
    print(f"   90th Percentile: {test_sharpe_dist['p90']:.3f}")
    print(f"   Skewness: {test_sharpe_dist['skewness']:.2f}")
    print(f"   Kurtosis: {test_sharpe_dist['kurtosis']:.2f}")
    
    # Confidence Interval
    ci_lower, ci_upper = calculate_confidence_interval(df_results['test_sharpe'])
    print(f"\n📏 95% Confidence Interval:")
    print(f"   Range: [{ci_lower:.3f}, {ci_upper:.3f}]")
    print(f"   Profitable: {'✓ YES' if ci_lower > 0 else '✗ NO'} (lower bound {'>' if ci_lower > 0 else '≤'} 0)")
    
    # Deflated Sharpe Ratio
    test_sharpes = df_results['test_sharpe'].values
    dsr = calculate_deflated_sharpe_ratio(test_sharpes)
    print(f"\n🔬 Deflated Sharpe Ratio (DSR):")
    print(f"   DSR: {dsr:.3f}")
    print(f"   Accounts for: {len(test_sharpes)} Monte Carlo trials")
    print(f"   Status: {'✓ SIGNIFICANT' if dsr >= 0.95 else '⚠ MARGINAL' if dsr >= 0.90 else '✗ NOT SIGNIFICANT'}")
    
    # MinTRL Analysis
    valid_min_trls = [r['min_trl'] for r in results if r['min_trl'] is not None and r['min_trl'] != float('inf')]
    if valid_min_trls:
        mean_min_trl = np.mean(valid_min_trls)
        actual_obs = df_results['test_observations'].mean()
        print(f"\n⏱ Minimum Track Record Length (MinTRL):")
        print(f"   Required: {mean_min_trl:.0f} observations (mean)")
        print(f"   Actual: {actual_obs:.0f} observations")
        print(f"   Status: {'✓ ADEQUATE' if actual_obs >= mean_min_trl else '✗ INSUFFICIENT'}")
    
    # Sample Size Adequacy
    mean_test_trades = df_results['test_trades'].mean()
    sample_assessment = assess_sample_size_adequacy(int(mean_test_trades))
    print(f"\n🔢 Sample Size Assessment:")
    print(f"   Test Trades (mean): {mean_test_trades:.0f}")
    print(f"   Adequacy: {sample_assessment['adequacy']}")
    print(f"   Confidence: {sample_assessment['confidence']}")
    print(f"   Recommended Minimum: {sample_assessment['recommended_minimum']}+")
    
    # ==================== BETTER OVERFITTING INDICATORS ====================
    
    print(f"\n" + "="*70)
    print("OVERFITTING INDICATORS")
    print("="*70)
    
    # 1. Test Sharpe Stability (Coefficient of Variation)
    test_sharpe_cv = (std_test / mean_test) if mean_test != 0 else float('inf')
    print(f"\n1. Test Sharpe Stability:")
    print(f"   Coefficient of Variation: {test_sharpe_cv:.2f}")
    print(f"   Interpretation: {'STABLE' if test_sharpe_cv < 0.5 else 'UNSTABLE' if test_sharpe_cv < 1.0 else 'HIGHLY UNSTABLE'}")
    print(f"   (Lower is better: <0.5 stable, 0.5-1.0 moderate, >1.0 unstable)")
    
    # 2. Walk-Forward Efficiency
    wf_efficiency = mean_test / mean_train if mean_train != 0 else 0
    print(f"\n2. Walk-Forward Efficiency:")
    print(f"   OOS Sharpe / IS Sharpe: {wf_efficiency:.1%}")
    print(f"   Interpretation: {'EXCELLENT' if wf_efficiency > 0.80 else 'GOOD' if wf_efficiency > 0.60 else 'ACCEPTABLE' if wf_efficiency > 0.40 else 'WEAK' if wf_efficiency > 0.25 else 'SEVERE OVERFIT'}")
    print(f"   (Expected: 25-80% for robust strategies)")
    
    # 3. Test Sharpe vs Baseline
    test_vs_baseline = (mean_test - config['baseline_sharpe']) / config['baseline_sharpe'] if config['baseline_sharpe'] != 0 else 0
    print(f"\n3. Test Sharpe vs Baseline:")
    print(f"   Difference: {test_vs_baseline:+.1%}")
    print(f"   Mean Test: {mean_test:.3f} vs Baseline: {config['baseline_sharpe']:.3f}")
    if abs(test_vs_baseline) < 0.15:
        print(f"   Interpretation: CONSISTENT (within 15%)")
    elif test_vs_baseline > 0.15:
        print(f"   Interpretation: UNEXPECTED - Test > Baseline (possible data issue)")
    else:
        print(f"   Interpretation: DEGRADED - Test << Baseline")
    
    # 4. Trade Count Analysis
    mean_train_trades = df_results['train_trades'].mean()
    mean_test_trades = df_results['test_trades'].mean()
    min_test_trades = df_results['test_trades'].min()
    print(f"\n4. Trade Count Analysis:")
    print(f"   Mean Training Trades: {mean_train_trades:.1f}")
    print(f"   Mean Testing Trades:  {mean_test_trades:.1f}")
    print(f"   Min Testing Trades:   {min_test_trades}")
    print(f"   Statistical Reliability: {'GOOD' if min_test_trades >= 30 else 'MARGINAL' if min_test_trades >= 15 else 'INSUFFICIENT'}")
    print(f"   (Need 30+ trades for statistical significance)")
    
    # 5. Consistency Analysis (% of runs with positive test Sharpe)
    positive_test_pct = (df_results['test_sharpe'] > 0).sum() / len(df_results)
    print(f"\n5. Consistency Analysis:")
    print(f"   Runs with positive test Sharpe: {(df_results['test_sharpe'] > 0).sum()}/{len(df_results)} ({positive_test_pct:.0%})")
    print(f"   Interpretation: {'HIGHLY CONSISTENT' if positive_test_pct >= 0.90 else 'CONSISTENT' if positive_test_pct >= 0.75 else 'MODERATE' if positive_test_pct >= 0.60 else 'INCONSISTENT'}")
    
    # 6. Sample Size Assessment
    print(f"\n6. Sample Size Assessment:")
    print(f"   Total runs: {len(df_results)}")
    print(f"   Statistical Power: {'ROBUST' if len(df_results) >= 1000 else 'ACCEPTABLE' if len(df_results) >= 100 else 'WEAK' if len(df_results) >= 50 else 'INSUFFICIENT' if len(df_results) >= 20 else 'ANECDOTAL'}")
    print(f"   Minimum required: 1000+ runs")
    if len(df_results) < 1000:
        print(f"   ⚠ WARNING: Results not statistically reliable with {len(df_results)} runs")
    
    # ==================== OVERALL ASSESSMENT ====================
    
    print(f"\n" + "="*70)
    print("OVERALL ROBUSTNESS ASSESSMENT")
    print("="*70 + "\n")
    
    # Calculate robustness score
    score_components = []
    
    # Test Sharpe stability (weight: 20%)
    if test_sharpe_cv < 0.5:
        score_components.append(('Sharpe Stability', 20, 20))
    elif test_sharpe_cv < 1.0:
        score_components.append(('Sharpe Stability', 20, 10))
    else:
        score_components.append(('Sharpe Stability', 20, 0))
    
    # Walk-forward efficiency (weight: 30%)
    if wf_efficiency > 0.80:
        score_components.append(('WF Efficiency', 30, 30))
    elif wf_efficiency > 0.60:
        score_components.append(('WF Efficiency', 30, 25))
    elif wf_efficiency > 0.40:
        score_components.append(('WF Efficiency', 30, 15))
    elif wf_efficiency > 0.25:
        score_components.append(('WF Efficiency', 30, 5))
    else:
        score_components.append(('WF Efficiency', 30, 0))
    
    # Test vs baseline (weight: 20%)
    if abs(test_vs_baseline) < 0.15:
        score_components.append(('Test vs Baseline', 20, 20))
    elif abs(test_vs_baseline) < 0.30:
        score_components.append(('Test vs Baseline', 20, 10))
    else:
        score_components.append(('Test vs Baseline', 20, 0))
    
    # Trade count (weight: 15%)
    if min_test_trades >= 30:
        score_components.append(('Trade Count', 15, 15))
    elif min_test_trades >= 15:
        score_components.append(('Trade Count', 15, 8))
    else:
        score_components.append(('Trade Count', 15, 0))
    
    # Consistency (weight: 15%)
    if positive_test_pct >= 0.90:
        score_components.append(('Consistency', 15, 15))
    elif positive_test_pct >= 0.75:
        score_components.append(('Consistency', 15, 12))
    elif positive_test_pct >= 0.60:
        score_components.append(('Consistency', 15, 8))
    else:
        score_components.append(('Consistency', 15, 0))
    
    total_score = sum(s[2] for s in score_components)
    max_score = sum(s[1] for s in score_components)
    
    print("Score Breakdown:")
    for name, max_pts, earned_pts in score_components:
        print(f"  {name}: {earned_pts}/{max_pts}")
    
    print(f"\nTotal Robustness Score: {total_score}/{max_score} ({total_score/max_score*100:.0f}%)")
    
    # Final decision
    if len(df_results) < 1000:
        decision = "INSUFFICIENT_SAMPLES"
        reason = f"Only {len(df_results)} runs (need 1000+ for validation)"
        recommendation = "Continue scaling to 1000+ runs before making conclusion"
    elif total_score >= 85:
        decision = "ROBUST_STRATEGY"
        reason = f"Score {total_score}/{max_score} - strong generalization"
        recommendation = "Strategy passes validation - ready for paper trading"
    elif total_score >= 70:
        decision = "PROCEED_WITH_CAUTION"
        reason = f"Score {total_score}/{max_score} - acceptable but not excellent"
        recommendation = "Strategy shows reasonable robustness - additional validation recommended"
    elif total_score >= 50:
        decision = "WEAK_ROBUSTNESS"
        reason = f"Score {total_score}/{max_score} - multiple concerns"
        recommendation = "Strategy shows weak generalization - use with caution or re-optimize"
    else:
        decision = "ABANDON_STRATEGY"
        reason = f"Score {total_score}/{max_score} - severe overfitting"
        recommendation = "Strategy fails validation - consider new hypothesis"
    
    print(f"\n✓ Decision: {decision}")
    print(f"  Reason: {reason}")
    print(f"  Recommendation: {recommendation}")
    
    # Save results
    output_data = {
        'strategy': 'Statistical Arbitrage Pairs Trading',
        'hypothesis_id': 5,
        'project_id': config['project_id'],
        'summary': {
            'sample_size': len(results),
            'successful_runs': len(results),
            'failed_runs': len(errors),
            'mean_train_sharpe': float(mean_train),
            'std_train_sharpe': float(std_train),
            'mean_test_sharpe': float(mean_test),
            'std_test_sharpe': float(std_test),
            'mean_degradation': float(mean_deg),
            'std_degradation': float(std_deg),
            'test_sharpe_cv': float(test_sharpe_cv),
            'wf_efficiency': float(wf_efficiency),
            'test_vs_baseline_pct': float(test_vs_baseline),
            'mean_train_trades': float(mean_train_trades),
            'mean_test_trades': float(mean_test_trades),
            'min_test_trades': int(min_test_trades),
            'positive_test_pct': float(positive_test_pct),
            'robustness_score': int(total_score),
            'max_score': int(max_score),
            'decision': decision,
            'reason': reason,
            'recommendation': recommendation
        },
        'detailed_results': results,
        'errors': errors
    }
    
    filename = f"walkforward_stat_arb_h5_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, 'w') as f:
        json.dump(output_data, f, indent=2, default=str)
    
    print(f"\n✓ Results saved to: {filename}")
    print("\n" + "="*70)
    print("MONTE CARLO WALK-FORWARD ANALYSIS COMPLETE")
    print("="*70)

In [ ]:
# ==================== DISPLAY HTML REPORT INLINE ====================

from IPython.display import HTML, display

if len(results) > 0:
    df_results = pd.DataFrame(results)
    
    # Recalculate metrics for HTML
    mean_train = df_results['train_sharpe'].mean()
    std_train = df_results['train_sharpe'].std()
    mean_test = df_results['test_sharpe'].mean()
    std_test = df_results['test_sharpe'].std()
    mean_deg = df_results['degradation'].mean()
    std_deg = df_results['degradation'].std()
    test_sharpe_cv = (std_test / mean_test) if mean_test != 0 else float('inf')
    wf_efficiency = mean_test / mean_train if mean_train != 0 else 0
    test_vs_baseline = (mean_test - config['baseline_sharpe']) / config['baseline_sharpe'] if config['baseline_sharpe'] != 0 else 0
    mean_train_trades = df_results['train_trades'].mean()
    mean_test_trades = df_results['test_trades'].mean()
    min_test_trades = df_results['test_trades'].min()
    positive_test_pct = (df_results['test_sharpe'] > 0).sum() / len(df_results)
    
    # Calculate advanced metrics for HTML
    test_psrs = [r['test_psr'] for r in results]
    psr_dist = analyze_mc_distribution(test_psrs)
    test_sharpe_dist = analyze_mc_distribution(df_results['test_sharpe'])
    ci_lower, ci_upper = calculate_confidence_interval(df_results['test_sharpe'])
    dsr = calculate_deflated_sharpe_ratio(df_results['test_sharpe'].values)
    test_sharpes = df_results['test_sharpe'].values
    
    # Calculate score components
    score_components = []
    if test_sharpe_cv < 0.5:
        score_components.append(('Sharpe Stability', 20, 20))
    elif test_sharpe_cv < 1.0:
        score_components.append(('Sharpe Stability', 20, 10))
    else:
        score_components.append(('Sharpe Stability', 20, 0))
    
    if wf_efficiency > 0.80:
        score_components.append(('WF Efficiency', 30, 30))
    elif wf_efficiency > 0.60:
        score_components.append(('WF Efficiency', 30, 25))
    elif wf_efficiency > 0.40:
        score_components.append(('WF Efficiency', 30, 15))
    elif wf_efficiency > 0.25:
        score_components.append(('WF Efficiency', 30, 5))
    else:
        score_components.append(('WF Efficiency', 30, 0))
    
    if abs(test_vs_baseline) < 0.15:
        score_components.append(('Test vs Baseline', 20, 20))
    elif abs(test_vs_baseline) < 0.30:
        score_components.append(('Test vs Baseline', 20, 10))
    else:
        score_components.append(('Test vs Baseline', 20, 0))
    
    if min_test_trades >= 30:
        score_components.append(('Trade Count', 15, 15))
    elif min_test_trades >= 15:
        score_components.append(('Trade Count', 15, 8))
    else:
        score_components.append(('Trade Count', 15, 0))
    
    if positive_test_pct >= 0.90:
        score_components.append(('Consistency', 15, 15))
    elif positive_test_pct >= 0.75:
        score_components.append(('Consistency', 15, 12))
    elif positive_test_pct >= 0.60:
        score_components.append(('Consistency', 15, 8))
    else:
        score_components.append(('Consistency', 15, 0))
    
    total_score = sum(s[2] for s in score_components)
    max_score = sum(s[1] for s in score_components)
    score_pct = total_score / max_score * 100
    
    # Decision
    if len(results) < 1000:
        decision = "INSUFFICIENT_SAMPLES"
        reason = f"Only {len(results)} runs (need 1000+ for validation)"
        recommendation = "Continue scaling to 1000+ runs before making conclusion"
        box_color = "#f39c12"
    elif total_score >= 85:
        decision = "ROBUST_STRATEGY"
        reason = f"Score {total_score}/{max_score} - strong generalization"
        recommendation = "Strategy passes validation - ready for paper trading"
        box_color = "#27ae60"
    elif total_score >= 70:
        decision = "PROCEED_WITH_CAUTION"
        reason = f"Score {total_score}/{max_score} - acceptable but not excellent"
        recommendation = "Strategy shows reasonable robustness - additional validation recommended"
        box_color = "#3498db"
    elif total_score >= 50:
        decision = "WEAK_ROBUSTNESS"
        reason = f"Score {total_score}/{max_score} - multiple concerns"
        recommendation = "Strategy shows weak generalization - use with caution or re-optimize"
        box_color = "#f39c12"
    else:
        decision = "ABANDON_STRATEGY"
        reason = f"Score {total_score}/{max_score} - severe overfitting"
        recommendation = "Strategy fails validation - consider new hypothesis"
        box_color = "#e74c3c"
    
    # Build detailed results rows
    results_rows = ""
    for result in results:
        results_rows += f"""
                <tr>
                    <td>{result['run']}</td>
                    <td>{result['train_start'].date()} to {result['train_end'].date()}</td>
                    <td>{result['test_start'].date()} to {result['test_end'].date()}</td>
                    <td>{result['train_sharpe']:.3f}</td>
                    <td>{result['test_sharpe']:.3f}</td>
                    <td>{result['degradation']*100:.1f}%</td>
                    <td>{result['train_trades']}</td>
                    <td>{result['test_trades']}</td>
                </tr>"""
    
    # Build score breakdown rows
    score_rows = ""
    for name, max_pts, earned_pts in score_components:
        score_rows += f"<tr><td>{name}</td><td>{max_pts}</td><td>{earned_pts}</td></tr>"
    
    # Generate HTML (to display inline)
    html = f"""
    <style>
        .mc-report {{ font-family: 'Segoe UI', Arial, sans-serif; background: white; padding: 30px; border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); }}
        .mc-report h1 {{ color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; margin-top: 0; }}
        .mc-report h2 {{ color: #34495e; margin-top: 25px; border-bottom: 2px solid #ecf0f1; padding-bottom: 8px; }}
        .mc-report .metric-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(220px, 1fr)); gap: 15px; margin: 15px 0; }}
        .mc-report .metric-card {{ background: #ecf0f1; padding: 15px; border-radius: 6px; border-left: 4px solid #3498db; }}
        .mc-report .metric-label {{ font-size: 11px; color: #7f8c8d; text-transform: uppercase; margin-bottom: 5px; }}
        .mc-report .metric-value {{ font-size: 28px; font-weight: bold; color: #2c3e50; }}
        .mc-report .metric-subtext {{ font-size: 12px; color: #95a5a6; margin-top: 5px; }}
        .mc-report table {{ width: 100%; border-collapse: collapse; margin: 15px 0; font-size: 13px; }}
        .mc-report th {{ background: #34495e; color: white; padding: 10px; text-align: left; }}
        .mc-report td {{ padding: 8px; border-bottom: 1px solid #ecf0f1; }}
        .mc-report tr:hover {{ background: #f8f9fa; }}
        .mc-report .status-good {{ color: #27ae60; font-weight: bold; }}
        .mc-report .status-warn {{ color: #f39c12; font-weight: bold; }}
        .mc-report .status-bad {{ color: #e74c3c; font-weight: bold; }}
        .mc-report .decision-box {{ background: {box_color}; color: white; padding: 25px; border-radius: 8px; margin: 20px 0; text-align: center; }}
        .mc-report .decision-box h2 {{ color: white; border: none; margin: 0 0 10px 0; }}
        .mc-report .progress-bar {{ width: 100%; background: #ecf0f1; height: 25px; border-radius: 12px; overflow: hidden; margin: 10px 0; }}
        .mc-report .progress-fill {{ background: linear-gradient(90deg, #3498db, #2ecc71); height: 100%; text-align: center; line-height: 25px; color: white; font-weight: bold; }}
        .mc-report ul {{ line-height: 1.8; }}
        .mc-report .dist-note {{ color: #7f8c8d; font-size: 14px; margin-bottom: 10px; font-style: italic; }}
    </style>
    
    <div class="mc-report">
        <h1>📊 Monte Carlo Walk-Forward Validation Report</h1>
        <p><strong>Strategy:</strong> Hypothesis 5 - Statistical Arbitrage Pairs Trading</p>
        <p><strong>Project ID:</strong> {config['project_id']} | <strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <h2>Configuration</h2>
        <ul>
            <li><strong>Pairs:</strong> {', '.join([p['name'] for p in config['pairs']])}</li>
            <li><strong>Date Range:</strong> {config['total_period']['start'].date()} to {config['total_period']['end'].date()}</li>
            <li><strong>Train/Test Split:</strong> {config['train_test_split']*100:.0f}% / {(1-config['train_test_split'])*100:.0f}%</li>
            <li><strong>Monte Carlo Runs:</strong> {len(results)} successful / {len(errors)} failed</li>
            <li><strong>Parameters:</strong> z_entry={config['parameters']['z_entry_threshold']}, z_exit={config['parameters']['z_exit_threshold']}, lookback={config['parameters']['lookback_period']}</li>
        </ul>
        
        <h2>Performance Summary</h2>
        <div class="metric-grid">
            <div class="metric-card">
                <div class="metric-label">Baseline Sharpe</div>
                <div class="metric-value">{config['baseline_sharpe']:.3f}</div>
                <div class="metric-subtext">From optimization</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Mean Training Sharpe</div>
                <div class="metric-value">{mean_train:.3f}</div>
                <div class="metric-subtext">± {std_train:.3f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Mean Testing Sharpe</div>
                <div class="metric-value">{mean_test:.3f}</div>
                <div class="metric-subtext">± {std_test:.3f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Mean Degradation</div>
                <div class="metric-value">{mean_deg*100:.1f}%</div>
                <div class="metric-subtext">± {std_deg*100:.1f}%</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Median Test PSR</div>
                <div class="metric-value">{psr_dist['median']:.3f}</div>
                <div class="metric-subtext">10th %ile: {psr_dist['p10']:.3f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Deflated Sharpe (DSR)</div>
                <div class="metric-value">{dsr:.3f}</div>
                <div class="metric-subtext">{len(test_sharpes)} trials</div>
            </div>
        </div>
        
        <h2>Overfitting Indicators</h2>
        <table>
            <tr>
                <th>Indicator</th>
                <th>Value</th>
                <th>Interpretation</th>
                <th>Status</th>
            </tr>
            <tr>
                <td>1. Test Sharpe Stability (CV)</td>
                <td>{test_sharpe_cv:.2f}</td>
                <td>{'STABLE' if test_sharpe_cv < 0.5 else 'MODERATE' if test_sharpe_cv < 1.0 else 'UNSTABLE'}</td>
                <td class="{'status-good' if test_sharpe_cv < 0.5 else 'status-warn' if test_sharpe_cv < 1.0 else 'status-bad'}">{'✓' if test_sharpe_cv < 0.5 else '⚠' if test_sharpe_cv < 1.0 else '✗'}</td>
            </tr>
            <tr>
                <td>2. Walk-Forward Efficiency</td>
                <td>{wf_efficiency:.1%}</td>
                <td>{'EXCELLENT' if wf_efficiency > 0.80 else 'GOOD' if wf_efficiency > 0.60 else 'ACCEPTABLE' if wf_efficiency > 0.40 else 'WEAK'}</td>
                <td class="{'status-good' if wf_efficiency > 0.60 else 'status-warn' if wf_efficiency > 0.40 else 'status-bad'}">{'✓' if wf_efficiency > 0.60 else '⚠' if wf_efficiency > 0.40 else '✗'}</td>
            </tr>
            <tr>
                <td>3. Test vs Baseline</td>
                <td>{test_vs_baseline:+.1%}</td>
                <td>{'CONSISTENT' if abs(test_vs_baseline) < 0.15 else 'DEGRADED' if test_vs_baseline < -0.15 else 'UNEXPECTED'}</td>
                <td class="{'status-good' if abs(test_vs_baseline) < 0.15 else 'status-warn' if abs(test_vs_baseline) < 0.30 else 'status-bad'}">{'✓' if abs(test_vs_baseline) < 0.15 else '⚠' if abs(test_vs_baseline) < 0.30 else '✗'}</td>
            </tr>
            <tr>
                <td>4. Trade Count (min test)</td>
                <td>{min_test_trades}</td>
                <td>{'GOOD' if min_test_trades >= 30 else 'MARGINAL' if min_test_trades >= 15 else 'INSUFFICIENT'}</td>
                <td class="{'status-good' if min_test_trades >= 30 else 'status-warn' if min_test_trades >= 15 else 'status-bad'}">{'✓' if min_test_trades >= 30 else '⚠' if min_test_trades >= 15 else '✗'}</td>
            </tr>
            <tr>
                <td>5. Consistency (positive runs)</td>
                <td>{positive_test_pct:.0%}</td>
                <td>{'HIGHLY CONSISTENT' if positive_test_pct >= 0.90 else 'CONSISTENT' if positive_test_pct >= 0.75 else 'MODERATE' if positive_test_pct >= 0.60 else 'INCONSISTENT'}</td>
                <td class="{'status-good' if positive_test_pct >= 0.75 else 'status-warn' if positive_test_pct >= 0.60 else 'status-bad'}">{'✓' if positive_test_pct >= 0.75 else '⚠' if positive_test_pct >= 0.60 else '✗'}</td>
            </tr>
            <tr>
                <td>6. Sample Size</td>
                <td>{len(results)} runs</td>
                <td>{'ROBUST' if len(results) >= 1000 else 'ACCEPTABLE' if len(results) >= 100 else 'WEAK' if len(results) >= 50 else 'INSUFFICIENT'}</td>
                <td class="{'status-good' if len(results) >= 1000 else 'status-warn' if len(results) >= 100 else 'status-bad'}">{'✓' if len(results) >= 1000 else '⚠' if len(results) >= 100 else '✗'}</td>
            </tr>
            <tr>
                <td>7. Probabilistic Sharpe Ratio</td>
                <td>{psr_dist['median']:.3f} (p10: {psr_dist['p10']:.3f})</td>
                <td>{'SIGNIFICANT' if psr_dist['p10'] >= 0.95 else 'MARGINAL' if psr_dist['p10'] >= 0.90 else 'NOT SIGNIFICANT'}</td>
                <td class="{'status-good' if psr_dist['p10'] >= 0.95 else 'status-warn' if psr_dist['p10'] >= 0.90 else 'status-bad'}">
                    {'✓' if psr_dist['p10'] >= 0.95 else '⚠' if psr_dist['p10'] >= 0.90 else '✗'}
                </td>
            </tr>
        </table>
        
        <h2>Distribution Analysis</h2>
        <p class="dist-note">Percentiles provide robust estimates for non-normal distributions. Use 10th percentile for conservative planning.</p>
        <table>
            <tr><th>Metric</th><th>10th %ile</th><th>Median</th><th>90th %ile</th><th>95% CI</th></tr>
            <tr>
                <td>Test Sharpe</td>
                <td>{test_sharpe_dist['p10']:.3f}</td>
                <td>{test_sharpe_dist['median']:.3f}</td>
                <td>{test_sharpe_dist['p90']:.3f}</td>
                <td>[{ci_lower:.3f}, {ci_upper:.3f}]</td>
            </tr>
            <tr>
                <td>PSR</td>
                <td>{psr_dist['p10']:.3f}</td>
                <td>{psr_dist['median']:.3f}</td>
                <td>{psr_dist['p90']:.3f}</td>
                <td>N/A</td>
            </tr>
        </table>
        
        <h2>Robustness Score</h2>
        <table>
            <tr><th>Component</th><th>Weight</th><th>Earned</th></tr>
            {score_rows}
            <tr style="background: #ecf0f1; font-weight: bold;">
                <td>TOTAL</td>
                <td>{max_score}</td>
                <td>{total_score}</td>
            </tr>
        </table>
        
        <div class="progress-bar">
            <div class="progress-fill" style="width: {score_pct}%">{score_pct:.0f}%</div>
        </div>
        
        <div class="decision-box">
            <h2>✓ DECISION: {decision}</h2>
            <p><strong>Reason:</strong> {reason}</p>
            <p><strong>Recommendation:</strong> {recommendation}</p>
        </div>
        
        <h2>Detailed Results (All {len(results)} Runs)</h2>
        <table>
            <tr>
                <th>Run</th>
                <th>Train Period</th>
                <th>Test Period</th>
                <th>Train Sharpe</th>
                <th>Test Sharpe</th>
                <th>Degradation</th>
                <th>Train Trades</th>
                <th>Test Trades</th>
            </tr>
            {results_rows}
        </table>
    </div>
    """
    
    # Display inline
    display(HTML(html))
    print(f"\n✓ Complete HTML report displayed above with all {len(results)} runs")
    
else:
    print("✗ No results to generate HTML report")